In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from group2_helpers import load_data_B, run_partition_tests_B, run_metamorphic_tests_B_bad, \
    visualize_bias_comparison_B, sensitive_columns_B
from group2_helpers import engineer_bias_features_B, load_onnx_model, predict_onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx
from pathlib import Path
from group2_helpers import print_disparate_impact_report_B


In [3]:
DATA_PATH_B   = Path("../data/synth_data_for_training.csv")
MODEL_B1_PATH = Path("model_1.onnx")  # BAD
MODEL_B2_PATH = Path("model_2.onnx")  # GOOD

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [4]:

# Make sure the output directory exists
MODEL_DIR_B = Path("")
MODEL_DIR_B.mkdir(parents=True, exist_ok=True)

# Load full data
df_B = pd.read_csv(DATA_PATH_B)
y_full_B = df_B["checked"].astype(int)
X_full_B = df_B.drop(columns=["checked"]).astype(np.float32)

# -------------------------
# BAD MODEL B: with bias features
# -------------------------
X_bad_B = engineer_bias_features_B(X_full_B)

X_train_bB, X_test_bB, y_train_bB, y_test_bB = train_test_split(
    X_bad_B, y_full_B, test_size=0.25, random_state=RANDOM_SEED
)

bad_model_B = GradientBoostingClassifier(
    n_estimators=200, learning_rate=0.05, max_depth=3, random_state=0
)
bad_model_B.fit(X_train_bB, y_train_bB)

y_pred_bB = bad_model_B.predict(X_test_bB)
print("[Subgroup B] BAD model accuracy:", accuracy_score(y_test_bB, y_pred_bB))

# Export BAD B to ONNX
initial_type_bad_B = [('X', FloatTensorType([None, X_bad_B.shape[1]]))]
bad_model_B_onnx = convert_sklearn(bad_model_B, initial_types=initial_type_bad_B)
onnx.save(bad_model_B_onnx, MODEL_DIR_B / "model_1.onnx")

# -------------------------
# GOOD MODEL B: drop sensitive / proxy columns
# -------------------------
X_good_B = X_full_B.drop(columns=[c for c in sensitive_columns_B if c in X_full_B.columns])

X_train_gB, X_test_gB, y_train_gB, y_test_gB = train_test_split(
    X_good_B, y_full_B, test_size=0.25, random_state=RANDOM_SEED
)

good_model_B = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("clf", GradientBoostingClassifier(
        n_estimators=200, learning_rate=0.05, max_depth=3, random_state=0
    )),
])

good_model_B.fit(X_train_gB, y_train_gB)
y_pred_gB = good_model_B.predict(X_test_gB)
print("[Subgroup B] GOOD model accuracy:", accuracy_score(y_test_gB, y_pred_gB))

# Export GOOD B to ONNX
initial_type_good_B = [('X', FloatTensorType([None, X_good_B.shape[1]]))]
good_model_B_onnx = convert_sklearn(good_model_B, initial_types=initial_type_good_B)
onnx.save(good_model_B_onnx, MODEL_DIR_B / "model_2.onnx")

print("\n[Subgroup B] Saved:")
print("  BAD  ->", MODEL_DIR_B / "model_1.onnx")
print("  GOOD ->", MODEL_DIR_B / "model_2.onnx")

[Subgroup B] GOOD model accuracy: 0.9275774826059456

[Subgroup B] Saved:
  BAD  -> model_1.onnx
  GOOD -> model_2.onnx


In [5]:
# Load raw data
X_raw_B, y_B, raw_cols_B = load_data_B()

print("\n[Subgroup B] Loading ONNX models...")
sessB1 = load_onnx_model(MODEL_B1_PATH)  # BAD B
sessB2 = load_onnx_model(MODEL_B2_PATH)  # GOOD B

# BAD B: original + engineered bias features
X_bad_view_B = engineer_bias_features_B(X_raw_B)
featB1 = list(X_bad_view_B.columns)

print(f"Model B1 (BAD) expects {sessB1.get_inputs()[0].shape[1]} features.")
print(f"We provide {len(featB1)} features.")

# GOOD B: drop B-sensitive columns
X_good_view_B = X_raw_B.drop(columns=[c for c in sensitive_columns_B if c in X_raw_B.columns])
featB2 = list(X_good_view_B.columns)

print(f"Model B2 (GOOD) expects {sessB2.get_inputs()[0].shape[1]} features.")
print(f"We provide {len(featB2)} features.")

# Base accuracy check
print("\n=== SUBGROUP B: BASE ACCURACY CHECK (correct feature views) ===")
predsB1 = predict_onnx(sessB1, X_bad_view_B, featB1)
print(f"model_B1 (BAD)  accuracy={accuracy_score(y_B, predsB1):.4f}, pos_rate={predsB1.mean():.3f}")

predsB2 = predict_onnx(sessB2, X_good_view_B, featB2)
print(f"model_B2 (GOOD) accuracy={accuracy_score(y_B, predsB2):.4f}, pos_rate={predsB2.mean():.3f}")

model_B1 (BAD)  accuracy=0.9522, pos_rate=0.058
model_B2 (GOOD) accuracy=0.9378, pos_rate=0.043


In [6]:
# Run on both models
run_partition_tests_B(X_bad_view_B, y_B, predsB1, model_label="model_B1 (BAD)")
run_partition_tests_B(X_good_view_B, y_B, predsB2, model_label="model_B2 (GOOD)")

  age_group=31-45: n=3723, acc=0.934, pos_rate=0.113
  age_group=46-60: n=6542, acc=0.964, pos_rate=0.014
  age_group= >=61: n=1814, acc=0.953, pos_rate=0.042

Partition on column: persoonlijke_eigenschappen_taaleis_voldaan (values: [0. 1. 2.])
  v=0.0: n=5043, acc=0.937, pos_rate=0.085
  v=1.0: n=7011, acc=0.965, pos_rate=0.034
  v=2.0: n= 591, acc=0.929, pos_rate=0.105

Partition on column: competentie_gedrevenheid_en_ambitie_tonen (values: [0. 1. 2.])
  v=0.0: n=10516, acc=0.955, pos_rate=0.053
  v=1.0: n=2128, acc=0.941, pos_rate=0.083
  v=2.0: group too small (n=1) – skipped.

Partition on column: relatie_kind_huidige_aantal (values: [0. 1. 2. 3. 4.])
  v=0.0: n=4443, acc=0.968, pos_rate=0.026
  v=1.0: n=6167, acc=0.947, pos_rate=0.070
  v=2.0: n=1782, acc=0.936, pos_rate=0.089
  v=3.0: n= 243, acc=0.926, pos_rate=0.086
  v=4.0: group too small (n=10) – skipped.

Partition on column: pla_historie_ontwikkeling (values: [0. 1.])
  v=0.0: n=4883, acc=0.924, pos_rate=0.128
  v=1.0: n=

In [7]:
print_disparate_impact_report_B(X_raw_B, y_B, predsB1, predsB2)


SUBGROUP B – DISPARATE IMPACT & BIAS METRICS

--- MODEL B1 (BAD) ---

  DI for Age: young (≤30) (1 vs 0):
    Group 0 size=12079, PPR=0.0486, FPR=0.0025
    Group 1 size=566, PPR=0.2544, FPR=0.0175
    Disparate Impact Ratio (1/0): 5.2353
    PPR difference (1-0):        +0.2058
    FPR difference (1-0):        +0.0150

  DI for Taaleis not met (1 vs 0):
    Group 0 size=7602, PPR=0.0397, FPR=0.0023
    Group 1 size=5043, PPR=0.0851, FPR=0.0044
    Disparate Impact Ratio (1/0): 2.1414
    PPR difference (1-0):        +0.0453
    FPR difference (1-0):        +0.0021

  DI for No ambition (1 vs 0):
    Group 0 size=2129, PPR=0.0827, FPR=0.0038
    Group 1 size=10516, PPR=0.0528, FPR=0.0029
    Disparate Impact Ratio (1/0): 0.6384
    PPR difference (1-0):        -0.0299
    FPR difference (1-0):        -0.0009
    → SEVERE disparity (below 80% rule).

  DI for Many children (>2) (1 vs 0):
    Group 0 size=12392, PPR=0.0571, FPR=0.0030
    Group 1 size=253, PPR=0.0909, FPR=0.0093
    Dis

In [8]:
metamorphic_results_B = run_metamorphic_tests_B_bad(X_raw_B, sessB1, featB1)


=== METAMORPHIC TESTS – SUBGROUP B BAD MODEL ===

[M-BAD] Metamorphic: young (≤30) → age 40
  Tested on 566 young examples.
  Prediction flips after age change: 4.24%

[M-BAD] Metamorphic: taaleis 0 → 1
  Tested on 1500 taaleis=0 examples.
  Prediction flips after taaleis 0→1: 1.60%

[M-BAD] Metamorphic: ambition 0 → 1
  Tested on 1500 ambition=0 examples.
  Prediction flips after ambition 0→1: 0.47%


In [9]:
print("\n" + "="*90)
print("GENERATING BIAS VISUALIZATION – SUBGROUP B")
print("="*90)
visualize_bias_comparison_B(X_raw_B, y_B, predsB1, predsB2)